In [17]:
import pandas as pd
from domain import preprocessing

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
filename = ("./beer_reviews.csv")
df = pd.read_csv(filename, sep=",")
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [19]:
# verifions combien de brasseries fabriquent une même bière
df.groupby(["beer_name"]).agg(nb_brewery = ('brewery_name','count')).reset_index()

,beer_name,nb_brewery
0,! (Old Ale),1
1,"""100""",5
2,"""100"" Pale Ale",1
3,"""12"" Belgian Golden Strong Ale",2
4,"""33"" Export",3
...,...,...
56852,Žamberecký Kanec Imperial Stout,2
56853,Ω-naught (Omeganaught),3
56854,“Chewy” Double IPA,1
56855,横須賀ビアサケ (Yokosuka Biasake),1


In [20]:
cols = ["review_time", "beer_beerid"]
df[cols].duplicated().sum()

47

In [21]:
mask = df["review_time"] == 1269285610
df[mask]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
59970,16866,The Bruery,1269285610,4.5,4.0,3.5,alpinebryant,Witbier,3.5,4.5,Orchard White,5.7,42433
59971,16866,The Bruery,1269285610,4.0,3.5,4.0,BeerGodz,Witbier,4.0,3.5,Orchard White,5.7,42433


In [22]:
df.shape

(1586614, 13)

In [ ]:
# brewery which produce the strongest beer
b = preprocessing.strongest_brewery(df)
{'brewery_name':b.to_dict()['brewery_name'].values()}

there are two brewery_id with no corresponding brewery_name.
So in the following, we consider only brewery_id for analysis.

To compute beer name to recommend, we focus only on beers which where noted at least one time after 2010.

In [24]:
df = preprocessing.convert_time(df)
df = preprocessing.beer_after_date(df, 2011)

In [25]:
df.shape

(882358, 15)

In [26]:
len(pd.unique(df.beer_beerid))

4367

In [27]:
most_beer = preprocessing.top_most_beers(df) 

In [28]:
most_beer

[77303, 71913, 77128]

In [29]:
df.columns

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv', 'beer_beerid',
       'date', 'year'],
      dtype='object')

In [30]:
column_note = ['review_overall','review_aroma', 'review_appearance','review_palate','review_taste']
x = df[column_note[1:]]
y = df[column_note[0]]
df[column_note].corr()

,review_overall,review_aroma,review_appearance,review_palate,review_taste
review_overall,1.000000,0.588001,0.481835,0.678185,0.767719
review_aroma,0.588001,1.000000,0.550420,0.602191,0.702118
review_appearance,0.481835,0.550420,1.000000,0.556734,0.535293
review_palate,0.678185,0.602191,0.556734,1.000000,0.719946
review_taste,0.767719,0.702118,0.535293,0.719946,1.000000


la colonne la plus correlée est review_taste avec un coefficient de 0.767719 en prenant en compte la base sur l'année 2012

In [31]:
#pour la regression lineaire 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x,y)
model.coef_

array([0.04022011, 0.03630855, 0.25153145, 0.53613001])

In [32]:
#top de beer en fonction de l'apparence et arome

most_beer_apparence_arome, data = preprocessing.top_most_beers_apparence_aroma(df) 

data[data.index.isin(most_beer_apparence_arome)]


,review_overall,review_aroma,review_palate,review_taste,review_appearance
beer_beerid,,,,,
77128,5.0,4.00,4.000,5.00,5.0
77045,4.5,5.00,5.000,5.00,3.0
76989,4.5,5.00,4.500,4.50,4.5
77203,4.5,5.00,4.500,4.50,4.5
44605,4.5,4.75,4.875,4.75,5.0
76947,4.5,4.50,4.000,4.50,5.0
77093,4.5,3.50,5.000,4.00,5.0
77205,4.0,5.00,4.500,3.50,3.0
77071,4.0,5.00,3.500,4.00,4.0


In [33]:
df.shape

(882358, 15)

In [ ]:
len(beer_id_2010)

In [35]:
df["beer_beerid"].nunique()

4367

In [36]:
df["brewery_id"].nunique()

1204

In [37]:
df["brewery_name"].nunique()

1200

In [38]:
mask = df["brewery_name"].isna()
df_na_name = df[mask]

In [39]:
df_na_name["brewery_id"].unique()

array([], dtype=int64)

In [40]:
df_na_name["brewery_id"].nunique()

0

In [41]:
mask = df["beer_abv"] == max(df["beer_abv"].unique())

In [42]:
df.loc[mask, ["brewery_id", "brewery_name", "beer_name", "beer_abv"]]

,brewery_id,brewery_name,beer_name,beer_abv
996149,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0
996150,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0
996151,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0
996152,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0
996153,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0
996154,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0
996155,15237,De Struise Brouwers,Black Damnation VI - Messy,39.0


In [43]:
df.isnull().sum().sort_values(ascending=False)

beer_abv              3252
review_profilename     178
brewery_id               0
brewery_name             0
review_time              0
review_overall           0
review_aroma             0
review_appearance        0
beer_style               0
review_palate            0
review_taste             0
beer_name                0
beer_beerid              0
date                     0
year                     0
dtype: int64

Nombre de fois qu'une bière est noté

In [44]:
beer_id_df = df.groupby(by="beer_beerid").agg(Count = ("beer_beerid","count"))
beer_id_df.reset_index()


,beer_beerid,Count
0,5,424
1,6,877
2,10,717
3,17,753
4,30,1736
...,...,...
4362,77313,1
4363,77314,1
4364,77315,1
4365,77316,1


In [ ]:
df[df["beer_beeid"]].isin(liste_beer_id) # à voir 

In [45]:
def beer_ratings(df):
    df = df.goupby(by="beer_beerid").agg(Count = ("beer_beerid","count"))
    return df.reset_index()

In [46]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,date,year
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159,2012-01-02 17:17:39,2012
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159,2011-10-19 02:25:15,2011
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159,2011-05-24 22:26:58,2011
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159,2010-11-22 19:35:03,2010
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159,2010-09-28 00:15:24,2010


In [55]:
df.groupby(by=["beer_beerid"]).agg({"beer_beerid":"count","review_taste":"mean"})

,beer_beerid,review_taste
beer_beerid,,
5,424,3.340802
6,877,3.645382
10,717,3.767782
17,753,3.142762
30,1736,4.264113
...,...,...
77313,1,3.000000
77314,1,4.000000
77315,1,3.500000
